In [9]:
import pandas as pd
import json

In [10]:
with open("all_dtypes.json", "r") as f:
    dtypes = json.load(f)

In [11]:
cnae = pd.read_csv("mergedfiles/id_cnae_fiscal_principal.csv", dtype=dtypes["id_cnae_fiscal_principal"])
cnae.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359 entries, 0 to 1358
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   cnae_fiscal_principal  1359 non-null   object
 1   descricao              1359 non-null   object
dtypes: object(2)
memory usage: 21.4+ KB


In [24]:
empresas = pd.read_csv("mergedfiles/empresas.csv", chunksize=10_000, dtype={"cnpj_basico": "str",
                                                                            "porte_empresa": "str",
                                                                            "natureza_juridica": "str"})
for chunk in empresas:
    chunk.info()
    break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   cnpj_basico                  10000 non-null  object
 1   razao_social                 10000 non-null  object
 2   natureza_juridica            10000 non-null  object
 3   qualificacoes_responsavel    10000 non-null  int64 
 4   capital_social               10000 non-null  object
 5   porte_empresa                10000 non-null  object
 6   ente_federativo_responsavel  3 non-null      object
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [25]:
for chunk in empresas:
    print(chunk.head())
    break

      cnpj_basico                                       razao_social  \
10000    41283621                       LIDER UNIFORMES SERVICE LTDA   
10001    41283622  SICREDI FUNDO DE INVESTIMENTO EM COTAS DE FI E...   
10002    41283623                       OLINTO CORREIA REPRESENTACAO   
10003    41283624               GHALA PET PRODUTOS PARA ANIMAIS LTDA   
10004    41283625   ALESSANDRA DAS GRACAS GOMES DA SILVA 07530354728   

      natureza_juridica  qualificacoes_responsavel capital_social  \
10000              2062                         49      110000,00   
10001              2224                         43           0,00   
10002              4014                         34           0,00   
10003              2062                         49      110000,00   
10004              2135                         50          10,00   

      porte_empresa ente_federativo_responsavel  
10000            01                         NaN  
10001            05                         NaN  
10